In [ ]:
# ------------------------------------------------------------------------------------------------- #
# Product:  Weather Map and SkewT plotter
# Model:    MEPS  
# Region:   Northern Europe
# Map:      Meteorological parameters based on surface, pressure, and severe weather conditions
# SkewT:    Vertical profile of tempwerature, dewpoint, wind, and vertical velocity. 
#           Includes a ground relative hodograph and a MUCAPE map.

# Process: 
# 1: Generate weather map
# 2: Click a location on the map if you want to retrieve a model sounding
# 3: Generate sounding (takes approx 45s)

# Future Updates
# Optimize speed, add more map products, add more features onto sounding.

# Questions or Suggestions?
# @FIN_storm on Twitter
# ------------------------------------------------------------------------------------------------- #

# IMPORTS
import cfgrib
import eccodes
from siphon.catalog import TDSCatalog

import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import cfgrib
import eccodes
from datetime import datetime

import metpy.constants as mconst
import metpy.calc as mpcalc
import metpy.plots as mpplots
from metpy.units import units
from metpy.plots import  SkewT, Hodograph
from metpy.units import units, pandas_dataframe_to_unit_arrays

import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import ArtistAnimation

import cartopy.crs as ccrs
import cartopy.feature as cfeature

## Map Plotter

In [ ]:
# Access road network file (optional if you want to plot roads on the map)
roadshp = gpd.read_file('C:tieosoiteverkko_0/tieosoiteverkkoLine.shp')

In [ ]:
# ACCESS MEPS DATA THROUGH NORWEGIAN METEOROLOGICAL SERVICE THREDDS SERVER
# This code only needs to be re-run if you want to change the initialization time

# Select model initialization date and time
# Available runs can be found here: https://thredds.met.no/thredds/catalog/mepslatest/catalog.html
# File names have this format: meps_det_2_5km_YYYYMMDDTHHZ.ncml
Idate = input('Enter Initialization Time: YYYYMMDD')
Itime = input('Enter Initialization Time: HH')
meps_data = TDSCatalog('https://thredds.met.no/thredds/catalog/mepslatest/catalog.xml?dataset=mepslatest/meps_det_2_5km_'+Idate+'T'+Itime+'Z.ncml')
meps_ds = meps_data.datasets[0]
ds = xr.open_dataset(meps_ds.access_urls['OPENDAP'])

# Collect universal data
time_ds = ds.metpy.parse_cf('time')
reftime = ds.metpy.parse_cf('forecast_reference_time')
projectionds = ds.metpy.parse_cf('projection_lambert')
x_crd = ds.metpy.parse_cf('x')
y_crd = ds.metpy.parse_cf('y')
hmsl = ds.metpy.parse_cf('height_above_msl')
clons, clats = np.meshgrid(x_crd, y_crd)
latnum = clats.shape[0]
lonnum = clons.shape[0]

FF00 = reftime

In [ ]:
# ---------------------------------- #
# PRODUCT SELECTION AND DATA RETRIEVAL
# ---------------------------------- #

Category = input('Select Category: a) Sfc b) Pressure c) Severe')

# SURFACE ---------------------------------------------------------------------------------------
if Category == 'a': 
    
    Product = input('Select Product: a) Temperature b) Dew Point c) Wind e) IR')
   
    if Product == 'a':
        tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
        uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
        vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')

    elif Product == 'b':
        rh_2m_ds = ds.metpy.parse_cf('relative_humidity_2m')
        tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
        uwnd_10m_ds = ds.metpy.parse_cf('x_wind_10m')
        vwnd_10m_ds = ds.metpy.parse_cf('y_wind_10m')

    elif Product == 'c':
        uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
        vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')
        wnd_gust_ds = ds.metpy.parse_cf('wind_speed_of_gust')
        wnd_speed_ds = ds.metpy.parse_cf('wind_speed')

    elif Product == 'e':
        IR_ds = ds.metpy.parse_cf('toa_outgoing_longwave_flux')

    else:
        print('Invalid Parameter Selected')
        

# PRESSURE ---------------------------------------------------------------------------------------
if Category == 'b': 
    Level = input('Enter Pressure Level: a) 1000mb b) 925mb c) 850mb d) 700mb e)500mb f)300mb')
    Product = input('Select Product: a) Wind b) (NA) Temperature c) Relative Humidity d) Dewpoint e) Vertical Velocity f) (NA) Vorticity')
    if Product == 'a':
        uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
        vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')
        geop_pl_ds = ds.metpy.parse_cf('geopotential_pl')
    if Product == 'c' or Product == 'd':
        rh_pl_ds = ds.metpy.parse_cf('relative_humidity_pl')
        geop_pl_ds = ds.metpy.parse_cf('geopotential_pl')
        uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
        vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl') 
    if Product == 'e':
        w_pl_ds = ds.metpy.parse_cf('upward_air_velocity_pl')
        geop_pl_ds = ds.metpy.parse_cf('geopotential_pl')
        uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
        vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')

    if Level == 'a':
        plevel = 1000
        lgp_min = 4
        lgp_max = 120
    elif Level == 'b':
        plevel = 925
        lgp_min = 4
        lgp_max = 200
    elif Level == 'c':
        plevel = 850
        lgp_min = 100
        lgp_max = 220
    elif Level =='d':
        plevel = 700
        lgp_min = 4
        lgp_max = 400
    elif Level == 'e':
        plevel = 500
        lgp_min = 480
        lgp_max = 600
    elif Level == 'f':
        plevel = 300
        lgp_min = 4
        lgp_max = 1200
    else: 
        print('Invalid Pressure Level')

# SEVERE WEATHER ---------------------------------------------------------------------------------------
if Category == 'c':
    option = input("a) Single Parameter b) Composite")
    if option == 'a': 
        Product = input("a) MUCAPE b) CIN c) LCL d) LFC e) PBL Thickness f) PW g) Lightning h) Cloud Tops i) Reflectivity j) 700-500 Lapse Rate")
        if Product == 'a':
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
        if Product == 'b':
            cin_ds = ds.metpy.parse_cf('atmosphere_convective_inhibition')
        if Product == 'c':
            lcl_ds = ds.metpy.parse_cf('lifting_condensation_level')
        if Product == 'd':
            lfc_ds = ds.metpy.parse_cf('atmosphere_level_of_free_convection')
        if Product == 'e':
            pbl_ds = ds.metpy.parse_cf('atmosphere_boundary_layer_thickness')
        if Product == 'f':
            pw_ds = ds.metpy.parse_cf('lwe_thickness_of_atmosphere_mass_content_of_water_vapor')
        if Product == 'g':
            ltg_ds = ds.metpy.parse_cf('lightning_index')
        if Product == 'h':
            cloud_top_ds = ds.metpy.parse_cf('cloud_top_altitude')
        if Product == 'i':
            rain_ds = ds.metpy.parse_cf('rainfall_amount')
            snow_ds = ds.metpy.parse_cf('snowfall_amount')
            graupel_ds = ds.metpy.parse_cf('graupelfall_amount')
        if Product == 'j':
            tmp_pl_ds = ds.metpy.parse_cf('air_temperature_pl')
            geop_ds = ds.metpy.parse_cf('geopotential_pl')
        if Product == 'k':
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            ap = ds.metpy.parse_cf('ap')
            b = ds.metpy.parse_cf('b')
            prs_sfc_ds = ds.metpy.parse_cf('surface_air_pressure')
            tmp_ml_ds = ds.metpy.parse_cf('air_temperature_ml')
            sh_ml_ds = ds.metpy.parse_cf('specific_humidity_ml')
    if option == 'b':
        Product = input("a) T/Wind/MUCAPE b) Sfc Vorticity & MUCAPE (NA) c) MUCAPE and 850mb Winds d) MUCAPE and 10m Wind" )
        if Product == 'a':
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
            uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
            vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')
        if  Product == 'b':
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
            vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')
        if Product == 'c':
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
            vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')
        if Product == 'd':
            cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
            uwnd_2m_ds = ds.metpy.parse_cf('x_wind_10m')
            vwnd_2m_ds = ds.metpy.parse_cf('y_wind_10m')

tmp_pl_ds = ds.metpy.parse_cf('air_temperature_pl')
tmp_2m_ds = ds.metpy.parse_cf('air_temperature_2m')
uwnd_pl_ds = ds.metpy.parse_cf('x_wind_pl')
vwnd_pl_ds = ds.metpy.parse_cf('y_wind_pl')

# ----------------------------------------------------------------------------------------------
# REFINES DATASET 
# e.g selects pressure level and performs calculations
# ----------------------------------------------------------------------------------------------

# SURFACE
if Category == 'a':
    if Product == 'a': # 2m Temperature
        tmp2m = tmp_2m_ds.sel(height1=2.0, method='nearest')
        uwnd2m = uwnd_2m_ds.sel(height7=10.0, method='nearest')
        vwnd2m = vwnd_2m_ds.sel(height7=10.0,  method='nearest')

    elif Product == 'b': # 2m Dewpoint
        rh2m = rh_2m_ds.sel(height1 = 2.0, method='nearest')
        tmp2m = tmp_2m_ds.sel(height1=2.0, method='nearest')
        td2m = mpcalc.dewpoint_from_relative_humidity(tmp2m, rh2m)
        uwnd10m = uwnd_10m_ds.sel(height7=10.0, method='nearest')
        vwnd10m = vwnd_10m_ds.sel(height7=10.0, method='nearest')

    elif Product == 'c': # 10m Wind
        uwnd2m = uwnd_2m_ds.sel(height7=10.0, method='nearest')
        vwnd2m = vwnd_2m_ds.sel(height7=10.0, method='nearest')
        wndG = wnd_gust_ds.sel(height7=10.0, method='nearest')
        wndS = wnd_speed_ds.sel(height7=10.0, method='nearest')

    elif Product == 'e': # Infrared Satellite
        ir1 = IR_ds.sel(top_of_atmosphere=0, method='nearest')
        # Calculate Temperature using Stefan Boltzmann Law
        ir = (( abs(ir1) / (5.67*(10**-8)))**(1/4)) - 273.15
        ir_30 = ir.where(ir <= -30)
        ir_3050 = ir_30.where(ir_30 >= -50)
        ir_5070 = ir.where(ir <= -50)

    elif Product == 'f': # Sounding Map
        tmp_pl = tmp_pl_ds.sel( method='nearest')
        rh_pl = rh_pl_ds.sel( method='nearest')
        td_pl = mpcalc.dewpoint_from_relative_humidity(tmp_pl* units.kelvin, rh_pl * units.kg / units.kg)


# PRESSURE
elif Category == 'b':
    if Product == 'a':
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, method='nearest')
        geop_pl = geop_pl_ds.sel(pressure=plevel, method='nearest')
        swnd_pl = mpcalc.wind_speed(uwnd_pl, vwnd_pl)
    if Product == 'b':
        tmp_pl = tmp_pl_ds.sel(pressure=plevel, method='nearest')
        geop_pl = geop_pl_ds.sel(pressure=plevel, method='nearest')
    if Product == 'c':
        rh_pl = rh_pl_ds.sel(pressure=plevel, method='nearest')
        geop_pl = geop_pl_ds.sel(pressure=plevel, method='nearest')
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, method='nearest')
    if Product == 'd':
        rh_pl = rh_pl_ds.sel(pressure=plevel, method='nearest')
        tmp_pl = tmp_pl_ds.sel(pressure=plevel, method='nearest')
        td_pl = mpcalc.dewpoint_from_relative_humidity(tmp_pl, rh_pl)
        geop_pl = geop_pl_ds.sel(pressure=plevel, method='nearest')
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, method='nearest')
    if Product == 'e':
        w_pl = w_pl_ds.sel(pressure=plevel, method='nearest')
        wp_pl = w_pl.where(w_pl > 0)
        wn_pl = w_pl.where(w_pl < 0)
        geop_pl = geop_pl_ds.sel(pressure=plevel, method='nearest')
        uwnd_pl = uwnd_pl_ds.sel(pressure=plevel, method='nearest')
        vwnd_pl = vwnd_pl_ds.sel(pressure=plevel, method='nearest')
        tmp_pl = tmp_pl_ds.sel(pressure=plevel, method='nearest')

    
# SEVERE
elif Category == 'c':
    if option == 'a':
        if Product == 'a':
            cape1 = cape_ds.sel(height0=0.0, method='nearest')
            cape = cape1.where((cape1 > 25))
        if Product == 'b':
            cin1 = cin_ds.sel(height0=0.0, method='nearest')
            cin = cin1.where((cin1 != 1000.0061)) 
        if Product == 'c':
            lcl = lcl_ds.sel(height0=0.0, method='nearest')
        if Product == 'd':
            lfc = lfc_ds.sel(height0=0.0, method='nearest')
        if Product == 'e':
            pbl = pbl_ds.sel(height0=0.0, method='nearest')
        if Product == 'f':
            pw = pw_ds.sel(surface=0, method='nearest')
        if Product =='g':
            ltg = ltg_ds.sel(surface=0.0, method='nearest')
        if Product =='h':
            cld_top = cloud_top_ds.sel(surface=0, method='nearest')
        if Product == 'i':
            rain_rate = rain_ds.sel(height0=0.0, method='nearest')
            rain_rate = rain_rate * 3600
            #dbz = 10log(aR**b)
            dbz_r = 10*np.log10(300*rain_rate**1.5)
        if Product == 'j':
            tmp_pl = tmp_pl_ds.sel( method='nearest')
            hght5 = geop_ds.sel( pressure=500, method='nearest') / 9.81
            hght7 = geop_ds.sel( pressure=700, method='nearest') / 9.81
            d57 = hght5 - hght7 
            lr57 = (tmp_pl.sel(pressure=700.00, method='nearest') - tmp_pl.sel(pressure=500.00, method='nearest')) / (d57/1000)
        if Product == 'k':
            tmp_ml = tmp_ml_ds.sel( method='nearest')
            sh_ml = tmp_ml_ds.sel( method='nearest')
            prs_sfc = prs_sfc_ds.sel( method='nearest')
            P = (( ap + b * prs_sfc) / 100)[::-1,0] * units.hPa
            g =  mconst.earth_gravity
            R = mconst.dry_air_gas_constant
            # Calculations using model level data
            Td = mpcalc.dewpoint_from_specific_humidity(P, tmp_ml, sh_ml * units.kg / units.kg)
           
            height = xr.full_like(P, fill_value=0) * units.meter
            height[0] = R*tmp_ml[0]/g * np.log(prs_sfc / P[0])
            
    if option == 'b':
        if Product == 'a':
            cape1 = cape_ds.sel(height0=0.0, method='nearest')
            cape = cape1.where((cape1 > 25))
            tmp2m = tmp_2m_ds.sel(height1=2.0, method='nearest')
            uwnd2m = uwnd_2m_ds.sel(height7=10.0, method='nearest')
            vwnd2m = vwnd_2m_ds.sel(height7=10.0, method='nearest')
        if Product == 'c':
            cape1 = cape_ds.sel(height0=0.0, method='nearest')
            cape_smooth = mpcalc.smooth_circular(cape1, 3, 3)
            cape = cape_smooth.where((cape_smooth > 25*units('J/kg')))
            uwnd_85 = uwnd_pl_ds.sel(pressure=850, method='nearest')
            vwnd_85 = vwnd_pl_ds.sel(pressure=850, method='nearest')
            wnd85 = mpcalc.wind_speed(uwnd_85, vwnd_85)
        if Product == 'd':
            cape1 = cape_ds.sel(height0=0.0, method='nearest')
            cape = cape1.where((cape1 > 25))
            uwnd2m = uwnd_2m_ds.sel(height7=10.0, method='nearest')
            vwnd2m = vwnd_2m_ds.sel(height7=10.0, method='nearest')

In [ ]:
#--------------------------------------------------------------#
# WEATHER MAP
# Interactive with a time slider. Keyboard shortcut: Arrow keys
# Click to select a location
# May take up to 5s to load each frame for complex products
#--------------------------------------------------------------#
%matplotlib widget

# Create Initial Plot
fig0 = plt.figure(figsize=(12, 8)) #dpi=200)
map_proj = ccrs.TransverseMercator(central_longitude=21, false_easting=1500000)
ax0 = fig0.add_subplot(1, 1, 1, projection=map_proj)

Sector = input('Sector Options: >Full >National >South >North >Custom')
if Sector == 'Full':
    ax0.set_extent((1, 36, 52, 74), crs=ccrs.PlateCarree())
    length = 4.5            # Wind barb length
    res = 25                # Wind barb spatial resolution
    fraction1 = 0.0154      # Colorbar Length
    xa = 0.12               # Watermark Position
    ya = 0.97               # Watermark Position
elif Sector == 'National':
    ax0.set_extent((15, 34, 58, 70.5), crs=ccrs.PlateCarree())
    length=5
    fraction1 = 0.0128
    res = 15
    xa = 0.2
    ya = 0.97
elif Sector == 'South':
    ax0.set_extent((17.4, 33.4, 59.6, 64.5), crs=ccrs.PlateCarree())
    length = 5
    fraction1 = 0.023
    res = 15
    xa = 0.125
    ya = 0.97
elif Sector == 'North':
    ax0.set_extent((17.4, 33.4, 64.5, 70.16), crs=ccrs.PlateCarree())
    fraction1 = 0.01857
    length=5
    res = 15
    xa = 0.15
    ya = 0.97
elif Sector == 'Custom':
    LocLat = 53.6179
    LocLon = 22.2542
    LatN = LocLat + 1.5
    LatS = LocLat - 1.5
    LonE = LocLon + 5
    LonW = LocLon - 5
    ax0.set_extent((LonW, LonE, LatS, LatN), crs=ccrs.PlateCarree())
    fraction1 = 0.023
    res = 10
    xa = 0.09
    ya = 0.95

# Map Features
lakes = cfeature.NaturalEarthFeature('physical', 'lakes', '10m', edgecolor='k', facecolor='none')
ax0.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.4)
ax0.add_feature(cfeature.BORDERS, linewidth=0.5)
ax0.add_feature(lakes, linewidth=0.1)

RoadChoice = input('Plot roads y/n (Caution this will slow down the plot)')
if RoadChoice == 'y':
    globe = ccrs.Globe(ellipse='GRS80')
    data_crs = ccrs.TransverseMercator(false_easting=500000, false_northing=0, 
                                        central_longitude=27.0, central_latitude=0, 
                                        scale_factor=0.9996, globe=globe)
    ax0.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='black', linewidths=0.7, alpha=0.88)
    ax0.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='gold', linewidths=0.4, alpha=1)
else:
    pass

# BUILD COLORMAPS
# -----------------------------------------------------------------------------------
cbar = None

# Radar Colormap
# --
color_db0020 = plt.cm.bone_r(np.linspace(0,0.55,73))
#color_db2060 = plt.cm.nipy_spectral(np.linspace(0.4,0.90,146))
color_db2030 = plt.cm.nipy_spectral(np.linspace(0.4,0.7,37))
color_db3050 = plt.cm.nipy_spectral(np.linspace(0.7, 0.9,110))
color_db6070 = plt.cm.gist_ncar(np.linspace(0.85,1,37))
color_db = np.vstack((color_db0020, color_db2030, color_db3050, color_db6070))
cmap_db = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_db)
# --     

# Temperature Colormap
# --
#color_TM2030 = plt.cm.Purples(np.linspace(0,1,43))
color_TM0020 = plt.cm.gist_rainbow_r(np.linspace(0,0.42,128))
#color_TM0020 = plt.cm.Blues(np.linspace(0.2,0.80,85))
color_T0020 = plt.cm.turbo(np.linspace(0.2,1,128))
color_T3040 = plt.cm.Reds(np.linspace(0.5,1,43))
color_T = np.vstack((color_TM0020, color_T0020))
cmap_t = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_T)
# --

# Dewpoint Colormap
# --
color_TdM0030 = plt.cm.BrBG(np.linspace(0,0.45,128))
color_Td0015 = plt.cm.Greens(np.linspace(0.1,1,64))
color_Td1520 = plt.cm.BrBG(np.linspace(0.7,1,21))
color_Td2030 = plt.cm.Purples(np.linspace(0.5,1,43))
color_td = np.vstack((color_TdM0030, color_Td0015, color_Td1520, color_Td2030))
cmap_td = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_td)
# --

# Wind Speed Colormap
# --
color_ws0025 = plt.cm.BuPu(np.linspace(0,0.9,160))
color_ws2540 = plt.cm.Wistia(np.linspace(0.4,1,96))
color_ws = np.vstack((color_ws0025, color_ws2540))
cmap_ws = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_ws)
# --     

# IR Satellite Colormap
# --
colors_ir1 = plt.cm.Greys(np.linspace(0.1,1,128))
colors_ir2 = plt.cm.GnBu(np.linspace(0.1, 0.7, 64))
colors_ir3 = plt.cm.magma(np.linspace(0,1, 64))
colors_ir = np.vstack((colors_ir3, colors_ir2, colors_ir1))
cmap_ir = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors_ir)
# --

# Pressure Level Wind Speed Colormap
# --
color_ws0070 = plt.cm.BuPu(np.linspace(0,0.6,149))
color_ws70120 = plt.cm.plasma(np.linspace(0.4,1,107))
color_wspl = np.vstack((color_ws0070, color_ws70120))
cmap_wspl = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_wspl)
# --    

# Vertical Velocity Colormap
# --
color_w1 = plt.cm.Greys_r(np.linspace(0,1,128))
color_w2 = plt.cm.inferno_r(np.linspace(0,1,128))
color_w = np.vstack((color_w1, color_w2))
cmap_w = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_w)
# --

# MUCAPE Colormap
# --
color_cp0050 = plt.cm.Greys(np.linspace(0.2,0.7,43))
color_cp50100 = plt.cm.Blues(np.linspace(0.2,0.7,42))
color_cp50300 = plt.cm.plasma_r(np.linspace(0,1,171))
color_cp = np.vstack((color_cp0050, color_cp50100, color_cp50300))
cmap_cp = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_cp)
# --     
# -----------------------------------------------------------------------------------

# Variables to store the clicked coordinates
lat_loc0 = None
lon_loc0 = None
def selected_coordinates(event):
    global lat_loc, lon_loc
    lon_loc0, lat_loc0 = event.xdata, event.ydata

    map_proj = ccrs.TransverseMercator(central_longitude=21, false_easting=1500000)
    degree_proj = ccrs.PlateCarree()
    data_transformed = degree_proj.transform_point(lon_loc0, lat_loc0, src_crs=map_proj)
    lon_loc, lat_loc = data_transformed[0], data_transformed[1]
    print(f"Selected: {lat_loc:.4f}N, {lon_loc:.4f}E")

# ------------------- #
# FUNCTION TO PLOT DATA
# ------------------- #

def plot_data(time_idx):
    global cbar
    ax0.clear()

    # Set extent of map
    if Sector == 'Full':
        ax0.set_extent((1, 36, 52, 74), crs=ccrs.PlateCarree())
    elif Sector == 'National':
        ax0.set_extent((15, 34, 58, 70.5), crs=ccrs.PlateCarree())
    elif Sector == 'South':
        ax0.set_extent((17.4, 33.4, 59.6, 64.5), crs=ccrs.PlateCarree())
    elif Sector == 'North':
        ax0.set_extent((17.4, 33.4, 64.5, 70.16), crs=ccrs.PlateCarree())
    elif Sector == 'Custom':
        LocLat = 53.6179
        LocLon = 22.2542
        LatN = LocLat + 1.5
        LatS = LocLat - 1.5
        LonE = LocLon + 5
        LonW = LocLon - 5
        ax0.set_extent((LonW, LonE, LatS, LatN), crs=ccrs.PlateCarree())
  

    # Base Map Features
    lakes = cfeature.NaturalEarthFeature('physical', 'lakes', '10m', edgecolor='k', facecolor='none')
    ax0.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.4)
    ax0.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax0.add_feature(lakes, linewidth=0.1)
    if RoadChoice == 'y':
        ax0.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='black', linewidths=0.7, alpha=0.88)
        ax0.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='gold', linewidths=0.4, alpha=1)
    else:
        pass

    # ------------------------------------------------------------------------------------
    # Plot Data
    global valid_time
    valid_time = time_ds.isel(time=time_idx)
    
        # SURFACE
    # ------------------
    if Category == 'a':
        if Product == 'a':
            MEPSprod = '2m Temperature (°C) and 10m Winds (kt)'
            mesh = ax0.pcolormesh(tmp2m.x, tmp2m.y, (tmp2m -273.15).isel(time=time_idx), transform=tmp2m.metpy.cartopy_crs, cmap=cmap_t, vmin=-30, vmax=30)
            tc = ax0.contour(tmp2m.metpy.x, tmp2m.metpy.y, tmp2m.isel(time=time_idx) -273.15, levels=0, colors='white', linestyles='--', linewidths=0.6,
                            transform=tmp2m.metpy.cartopy_crs, antialiased=True)
            #ax0.streamplot(uwnd2m.x, uwnd2m.y, uwnd2m, vwnd2m, transform=uwnd2m.metpy.cartopy_crs, density=4)
            ax0.barbs(clons[::res,::res],clats[::res,::res],(uwnd2m.isel(time=time_idx)[::res,::res].values)*1.94384,(vwnd2m.isel(time=time_idx)[::res,::res].values)*1.94384,
                    transform=uwnd2m.metpy.cartopy_crs,color='black',length=5, alpha=0.8, linewidth=0.2)

        if Product == 'b':
            MEPSprod = '2m Dewpoint (°C) and 10 Winds (kt)'
            mesh = ax0.pcolormesh(td2m.x, td2m.y, td2m.isel(time=time_idx), transform=td2m.metpy.cartopy_crs, cmap=cmap_td, vmin=-30, vmax=30)
            ax0.barbs(clons[::res,::res],clats[::res,::res],((uwnd10m.isel(time=time_idx))[::res,::res].values)*1.94384,((vwnd10m.isel(time=time_idx))[::res,::res].values)*1.94384,
                    transform=uwnd10m.metpy.cartopy_crs,color='black',length=6, alpha=0.8, linewidth=0.3)

        if Product == 'c':
            MEPSprod = '10m Wind Speed (m/s, shaded), Wind Gusts (m/s, contour)' 
            #mesh = ax.contourf(wndG.x, wndG.y, wndG, transform=wndG.metpy.cartopy_crs, levels=(20,22,24,26,28,30,32,34,36,38,40), cmap='Reds', alpha=0.8 ) 
            mesh = ax0.pcolormesh(wndS.x, wndS.y, wndS.isel(time=time_idx), transform=wndS.metpy.cartopy_crs, cmap=cmap_ws, vmin=0, vmax=40)
            ax0.barbs(clons[::res,::res],clats[::res,::res],(uwnd2m.isel(time=time_idx)[::res,::res].values)*1.94384,(vwnd2m.isel(time=time_idx)[::res,::res].values)*1.94384,
                    transform=uwnd2m.metpy.cartopy_crs,color='black',length=5, alpha=0.8, linewidth=0.4)
            wndGs = mpcalc.smooth_circular(wndG.isel(time=time_idx), 3, 2)
            wsp = ax0.contour(wndGs.x, wndGs.y, wndGs, levels=(15,20,25,30,35,40), cmap=plt.cm.Reds, vmin=0, vmax=40, linewidths=1.5,
                    transform=wndS.metpy.cartopy_crs, antialiased=True)
            plt.clabel(wsp, fontsize=15, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True,
                        use_clabeltext=True)
        
        if Product == 'e': # Infrared Satellite
            MEPSprod = 'IR Simulated Satellite (°C)'
            mesh = ax0.pcolormesh(ir.x, ir.y, ir.isel(time=time_idx), transform=ir.metpy.cartopy_crs, cmap=cmap_ir, vmin=-70, vmax=10)


    # PRESSURE
    # ------------------
    elif Category == 'b':
        if Product == 'a':
            MEPSprod = str(plevel)+'hPa Winds (kt, shaded), Geopotential Height (dam, contours)'
            mesh = ax0.contourf(swnd_pl.x, swnd_pl.y, swnd_pl.isel(time=time_idx)*1.94384, transform=swnd_pl.metpy.cartopy_crs, 
                            levels=(10,20,30,40,50,60,70,80,90,100,110,120), cmap=cmap_wspl, vmin=0, vmax=120)
            ax0.barbs(clons[::res,::res],clats[::res,::res],(uwnd_pl.isel(time=time_idx)[::res,::res].values)*1.94384,(vwnd_pl.isel(time=time_idx)[::res,::res].values)*1.94384,
                    transform=uwnd_pl.metpy.cartopy_crs,color='black',length=length, alpha=0.8, linewidth=0.2)
            Geo_Pot = ax0.contour(geop_pl.x, geop_pl.y, geop_pl.isel(time=time_idx) / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')
            plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True)
        if Product == 'c':
            MEPSprod = str(plevel)+'hPa Relative Humidity (%, shaded)'
            mesh = ax0.pcolormesh(rh_pl.x, rh_pl.y, rh_pl.isel(time=time_idx) * 100, transform=rh_pl.metpy.cartopy_crs, cmap='BrBG', vmin=0, vmax=100, alpha=0.8)
            ax0.barbs(clons[::res,::res],clats[::res,::res],(uwnd_pl.isel(time=time_idx)[::res,::res].values)*1.94384,(vwnd_pl.isel(time=time_idx)[::res,::res].values)*1.94384,
                    transform=uwnd_pl.metpy.cartopy_crs,color='black',length=length, alpha=0.8, linewidth=0.2)  
            Geo_Pot = ax0.contour(geop_pl.x, geop_pl.y, geop_pl.isel(time=time_idx) / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')
            plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 
        if Product == 'd':
            MEPSprod = str(plevel)+'hPa Dew Point (°C, shaded)'
            mesh = ax0.pcolormesh(td_pl.x, td_pl.y, td_pl.isel(time=time_idx), transform=td_pl.metpy.cartopy_crs, cmap=cmap_td, vmin=-30, vmax=30)
            ax0.barbs(clons[::res,::res],clats[::res,::res],(uwnd_pl.isel(time=time_idx)[::res,::res].values)*1.94384,(vwnd_pl.isel(time=time_idx)[::res,::res].values)*1.94384,
                    transform=uwnd_pl.metpy.cartopy_crs,color='black',length=length, alpha=0.8, linewidth=0.2)
            Geo_Pot = ax0.contour(geop_pl.x, geop_pl.y, geop_pl.isel(time=time_idx) / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')  
            plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 
        if Product == 'e':
            MEPSprod = str(plevel)+'hPa Verical Velocity'
            mesh = ax0.pcolormesh(w_pl.x, w_pl.y, w_pl.isel(time=time_idx), transform = w_pl.metpy.cartopy_crs, cmap=cmap_w, vmin=-3, vmax=3)
            Geo_Pot = ax0.contour(geop_pl.x, geop_pl.y, geop_pl.isel(time=time_idx) / 98.0665, levels=np.arange(lgp_min, lgp_max, 4), transform=geop_pl.metpy.cartopy_crs, colors='k')  
            plt.clabel(Geo_Pot, Geo_Pot.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 

    # T-STORMS
    # ------------------
    elif Category == 'c':
        if option == 'a':
            if Product == 'a':
                MEPSprod = 'MUCAPE (J/kg)'
                mesh = ax0.pcolormesh(cape.x, cape.y, cape.isel(time=time_idx), transform = cape.metpy.cartopy_crs, cmap=cmap_cp, vmin=0, vmax=3000)
            if Product == 'b':
                MEPSprod = 'CIN (J/Kg)'
                mesh = ax0.pcolormesh(cin.x, cin.y, cin.isel(time=time_idx), transform=cin.metpy.cartopy_crs, cmap='Greys', vmin=0, vmax=1000)
            if Product == 'c':
                MEPSprod = 'LCL (m)'
                mesh = ax0.pcolormesh(lcl.x, lcl.y, lcl.isel(time=time_idx), transform=lcl.metpy.cartopy_crs, cmap='Greens_r', vmin=0, vmax=4000)
            if Product == 'd':
                MEPSprod = 'LFC (m)'
                mesh = ax0.pcolormesh(lfc.x, lfc.y, lfc.isel(time=time_idx), transform=lfc.metpy.cartopy_crs, cmap='Greens_r', vmin=0, vmax=5000)
            if Product == 'e':
                MEPSprod = 'PBL Depth (m)'
                mesh = ax0.pcolormesh(pbl.x, pbl.y, pbl.isel(time=time_idx), transform=pbl.metpy.cartopy_crs, cmap='Oranges', vmin=0, vmax=3000)
            if Product == 'f':
                MEPSprod = 'Precipitable Water'
                mesh = ax0.pcolormesh(pw.x, pw.y, pw.isel(time=time_idx), transform=pw.metpy.cartopy_crs, cmap='BrBG', vmin=0, vmax=50)
                ax0.contour(pw.x, pw.y, pw.isel(time=time_idx), transform=pw.metpy.cartopy_crs, colors='purple', levels=(12,25,75,100), linewidths=0.5)
            if Product == 'g':
                MEPSprod = 'Lightning Strikes'
                mesh = ax0.pcolormesh(ltg.x, ltg.y, ltg.isel(time=time_idx), transform=ltg.metpy.cartopy_crs, cmap='RdPu')
            if Product == 'h':
                MEPSprod = 'Cloud Top Heights (m)'
                mesh = ax0.pcolormesh(cld_top.x, cld_top.y, cld_top.isel(time=time_idx), transform=cld_top.metpy.cartopy_crs, cmap='inferno_r', vmin=0, vmax=12000)
            if Product == 'i':
                MEPSprod = 'Reflectivity (dBz)'
                mesh = ax0.pcolormesh(dbz_r.x, dbz_r.y, dbz_r.isel(time=time_idx), transform=dbz_r.metpy.cartopy_crs, cmap=cmap_db, vmin=0, vmax=70)
            if Product == 'j':
                MEPSprod = '700 - 500mb Lapse Rate'
                lr57_s = mpcalc.smooth_circular(lr57.isel(time=time_idx), 4, 2)
                mesh = ax0.contour(lr57.x, lr57.y, lr57_s, transform=lr57.metpy.cartopy_crs, levels=(np.arange(5,10.5,0.5)), cmap='Reds', vmin=0, vmax=10)
                plt.clabel(mesh, mesh.levels[1::2], fontsize=10, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True) 
        if option == 'b':
            if Product == 'a':
                MEPSprod = '2m Temperature and MUCAPE'
                #cape_smooth = mpcalc.smooth_circular(cape, 3, 2)
                mesh = ax0.pcolormesh(tmp2m.x, tmp2m.y, tmp2m.isel(time=time_idx) -273.15, transform=tmp2m.metpy.cartopy_crs, cmap=cmap_t, vmin=-30, vmax=30, alpha=0.85)
                ax0.contourf(cape.x, cape.y, cape.isel(time=time_idx), transform = cape.metpy.cartopy_crs, cmap=cmap_cp, 
                            levels=(150,200,300,400,500,600,700,1000,1500,2000,2500,3000), 
                            alpha=0.7, vmin=0, vmax=3000)
                tc = ax0.contour(tmp2m.metpy.x, tmp2m.metpy.y, tmp2m.isel(time=time_idx) -273.15, levels=0, colors='grey', linestyles='--', linewidths=0.6,
                                transform=tmp2m.metpy.cartopy_crs, antialiased=True)
                #ax.streamplot(uwnd2m.x, uwnd2m.y, uwnd2m, vwnd2m, transform=uwnd2m.metpy.cartopy_crs, density=4)
                ax0.barbs(clons[::res,::res],clats[::res,::res],(uwnd2m.isel(time=time_idx)[::res,::res].values)*1.94384,(vwnd2m.isel(time=time_idx)[::res,::res].values)*1.94384,
                        transform=uwnd2m.metpy.cartopy_crs,color='black',length=5, alpha=0.8, linewidth=0.2)
            if Product == 'c':
                MEPSprod = 'MUCAPE (shaded, J/Kg) and 850mb Winds (contour, m/s)' 
                mesh = ax0.pcolormesh(cape.x, cape.y, cape.isel(time=time_idx), transform = cape.metpy.cartopy_crs, cmap=cmap_cp, vmin=0, vmax=3000)
                wnd85_smooth = mpcalc.smooth_circular(wnd85.isel(time=time_idx), 4, 7)
                contour85 = ax0.contour(wnd85.x, wnd85.y, wnd85_smooth, transform=wnd85.metpy.cartopy_crs, colors='k', linewidths=1.5, levels=(20,30,40,50,60))
                ax0.clabel(contour85, fontsize=15, colors='k', inline=1, inline_spacing=15, fmt='%i', rightside_up=True, use_clabeltext=True)

                ax0.quiver(clons[::res, ::res], clats[::res,::res], uwnd_85.isel(time=time_idx)[::res,::res].values, vwnd_85.isel(time=time_idx)[::res,::res].values, transform=uwnd_85.metpy.cartopy_crs,
                        width=0.001)
            if Product == 'd':
                MEPSprod = 'MUCAPE (shaded, J/Kg) and Surface Streamlines'
                mesh = ax0.pcolormesh(cape.x, cape.y, cape.isel(time=time_idx), transform = cape.metpy.cartopy_crs, cmap=cmap_cp, vmin=0, vmax=3000)
                ax0.streamplot(uwnd2m.x, uwnd2m.y, uwnd2m.isel(time=time_idx), vwnd2m.isel(time=time_idx), transform=uwnd2m.metpy.cartopy_crs, density=4)

    plt.title('MEPS '+((FF00.dt.strftime('%HZ')).values)+' | Valid '+((valid_time.dt.strftime('%Y-%m-%d %HZ')).values), size=10, loc='right')
    plt.title('MEPS || '+str(MEPSprod), size=10, loc='left')
    plt.text(xa, ya, '@FIN_storm', size=25, alpha=0.3,weight='bold', 
     horizontalalignment='center',
     verticalalignment='center',
     transform = ax0.transAxes)


    # Remove the previous colorbar if it exists
    if cbar is not None:
        cbar.remove()
    # Add colorbar
    cbar = plt.colorbar(mesh, orientation='horizontal',fraction=fraction1,pad=0.0,aspect=60,extendrect=True)
    # Connect the click event to the store_clicked_coordinates function
    fig0.canvas.mpl_connect('button_press_event', selected_coordinates)
    #plt.subplots_adjust(left=0.01, bottom=0.01, right=0.95, top=0.95, wspace=0.05, hspace=0.02)
    plt.draw()


time_slider = widgets.IntSlider(value=0, min=0, max=65, step=1, description="Time Index")
t_readout = widgets.Label()

interactive_plot = widgets.interactive(plot_data, time_idx=time_slider)
display(interactive_plot)

## Sounding Plotter

In [ ]:
# Access Meteorological Parameters
tmp_ml = ds.metpy.parse_cf('air_temperature_ml')
sh_ml = ds.metpy.parse_cf('specific_humidity_ml')
uwnd_ml = ds.metpy.parse_cf('x_wind_ml')
vwnd_ml = ds.metpy.parse_cf('y_wind_ml')
w_ml = ds.metpy.parse_cf('upward_air_velocity_ml')
prs_sfc = ds.metpy.parse_cf('surface_air_pressure')
tmp_2m = ds.metpy.parse_cf('air_temperature_2m')
rh_2m = ds.metpy.parse_cf('relative_humidity_2m')
cape_ds = ds.metpy.parse_cf('specific_convective_available_potential_energy')
# Constants required to convert convert hybrid to pressure vertical coordinate system
ap = ds.metpy.parse_cf('ap')
b = ds.metpy.parse_cf('b')

# Access model coordinate parameters
lon = ds.metpy.parse_cf('longitude')
lat = ds.metpy.parse_cf('latitude')

In [ ]:
# Automated Time and Location Selection
# Based on location selected on map with corresponding data and time
# Date
Year = np.datetime_as_string(valid_time, unit='Y')
Month = np.datetime_as_string(valid_time, unit='M').split('-')[1]
Day = np.datetime_as_string(valid_time, unit='D').split('-')[2]
Time = np.datetime_as_string(valid_time, unit='h').split('T')[1]
dt = str(Year)+'-'+str(Month)+'-'+str(Day)+'T'+str(Time)+':00:00.000000000'
# Location Selection
lat_s = lat_loc
lon_s = lon_loc

# Find array index closest to location
location = (lon_s, lat_s)
# calculate distance between center lat/lon and analog cases (Haversine)
# Code retrieved from: https://github.com/nixoncameronj/HodoMap
def haversine(lon_arr, lat_arr, lon_point, lat_point): 
    distance = 6371*(2*np.arcsin(np.sqrt(np.sin((np.radians(lat_arr)-np.radians(lat_point))/2)**2 + np.cos(np.radians(lat_point))
                                          * np.cos(np.radians(lat_arr)) * np.sin((np.radians(lon_arr)-np.radians(lon_point))/2)**2)))
    return distance
location_distance = haversine(lon,lat,location[0],location[1])
location_index = np.where(location_distance == np.min(location_distance))

In [ ]:
# APPLY TIME AND SPATIAL RESTRICTIONS TO DATA SETS
# PERFORM CALCULATIONS 
# Note: Requires ~20s

P0 = prs_sfc.sel(time=dt, method='nearest')[0, location_index[0], location_index[1]]
P01 = P0[0,0] * units.Pa
P = (( ap + b * P0) / 100)[::-1,0,0] * units.hPa
T = tmp_ml.sel(time=dt, method='nearest')[:, location_index[0], location_index[1]][::-1,0,0].metpy.convert_units('degC')
#T2m = tmp_2m.sel(time=dt, method='nearest')[0, location_index[0], location_index[1]][0,0].metpy.convert_units('degC')
SH = sh_ml.sel(time=dt, method='nearest')[:, location_index[0], location_index[1]][::-1,0,0]
#RH2m = rh_2m.sel(time=dt, method='nearest')[0, location_index[0], location_index[1]][0,0]
uwnd = uwnd_ml.sel(time=dt, method='nearest')[:, location_index[0], location_index[1]][::-1,0,0].metpy.convert_units('kt')
vwnd = vwnd_ml.sel(time=dt, method='nearest')[:, location_index[0], location_index[1]][::-1,0,0].metpy.convert_units('kt')
w = w_ml.sel(time=dt, method='nearest')[:, location_index[0], location_index[1]][::-1,0,0]

cape1 = cape_ds.sel(height0=0.0, time=dt, method='nearest')
cape = cape1.where((cape1 > 25))

g =  mconst.earth_gravity
R = mconst.dry_air_gas_constant

# Parameter Calculations
Td = mpcalc.dewpoint_from_specific_humidity(P, T, SH * units.kg / units.kg)
Tw = mpcalc.wet_bulb_temperature(P, T, Td)
Te = mpcalc.equivalent_potential_temperature(P, T, Td)

# Modify Vertical coordinates
height = xr.full_like(P, fill_value=0) * units.meter
height[0] = R*T[0]/g * np.log(P01 / P[0])

for hybrid in range(1,65):
    height[hybrid] = ((R*T[hybrid]/g) * np.log(P[hybrid - 1]/P[hybrid])).metpy.convert_units('m') + height[hybrid - 1]

stdhgt = [500,1000,2000,3000,6000,8000,10000]
hgtkm = [.5,1,2,3,6,8,10]
stdprs = np.interp(stdhgt, height, P)

In [ ]:
# --------------------------------------
# SOUNDING
# --------------------------------------
%matplotlib inline

# Set up Figure
fig = plt.figure(figsize=(20, 12))

# SKEWT
# ---------------------#
# ---------------------#
skew = SkewT(fig, rotation=30, aspect=110, rect=(0,0,1,1))

# Add basic skewT lines
skew.plot_dry_adiabats(linewidth=0.3)
skew.plot_moist_adiabats(linewidth=0.3)
skew.plot_mixing_lines(linewidth=0.2)
skew.ax.axvline(0, color='c', linestyle='--', linewidth=0.4)
skew.ax.set_ylim(1020, 300)
skew.ax.set_xlim(-30, 35)
# Mark Geometric Heights
for z in range(7):
  trans, _, _ = skew.ax.get_yaxis_text1_transform(0)
  skew.ax.text(0.00, stdprs[z], '--'+str(hgtkm[z])+'km', fontsize=10, va='center',ha='left', transform=trans, color='red', weight='bold')

# Mark Dendritic Growth Zone
skew.ax.axvline(-12, color='gold', linestyle='-', linewidth=0.8)
skew.ax.axvline(-17, color='gold', linestyle='-', linewidth=0.8)

# SkewT Data
skew.plot(P, T, 'tab:red', linewidth=2)
skew.plot(P, Td, 'tab:green', linewidth=1)
skew.plot(P, Tw, 'tab:cyan', linewidth=1)
# Wind Barbs
interval = np.logspace(8, 3) * units.hPa
idx = mpcalc.resample_nn_1d(P, interval)
skew.plot_barbs(P[::-3], uwnd[::-3], vwnd[::-3], y_clip_radius=0.01)
# Parcel Calculatiions
# -------------------#
# Surface Based
SB_parcel = mpcalc.parcel_profile(P, T[0], Td[0])
skew.plot(P, SB_parcel, 'k',linestyle='--', lw=0.5)
# Most Unstable
mu_p, mu_t, mu_td, _ = mpcalc.most_unstable_parcel(P, T, Td)
if P[0] == mu_p:
    if T[0] == mu_t:
       T = T
       mup = P
    elif T[0] != mu_t:
         T[0] = mu_t
         mup = P
elif P[0] != mu_p:
    mup = np.delete(P, np.s_[0:int(np.where(P==mu_p)[0])]) * units.hPa
    mup = mup[:(len(mup))]
MU_parcel = mpcalc.parcel_profile(mup, mu_t, mu_td)
skew.plot(mup, MU_parcel.metpy.convert_units('degC'), 'brown',linestyle='--', dashes=(7,7),lw=0.8)
# Mixed Layer
# DCAPE
Te_array = Te.to_masked_array()
te_index = np.argmin(Te_array)
dc_tw = Tw[te_index]
dc_p = P[0:te_index+1]
dc_p= dc_p[::-1]
dcape_parcel = mpcalc.moist_lapse(dc_p, float(dc_tw)*units('degC'))
skew.plot(dc_p, dcape_parcel, 'orange', linestyle='--', dashes=(7,7), lw=0.8)

# Calculate and plot LFC, LCL, and EL for a surface-based parcel
lcl_P, lcl_T = mpcalc.lcl(P[0], T[0], Td[0])
lfc_P, lfc_T = mpcalc.lfc(P, T, Td)
el_P, el_T = mpcalc.el(P, T, Td)

plt.text((0.85),(lcl_P), s='LCL', weight='bold', color='green', alpha = 0.9, fontsize=20, ha='left', transform=skew.ax.get_yaxis_transform())
plt.text((0.85), (lfc_P), s='LFC', weight='bold', color='gold', alpha = 0.9, fontsize=20, ha='left',transform=skew.ax.get_yaxis_transform())
plt.text((0.85), (el_P), s='EL', weight='bold', color='pink', alpha = 0.9, fontsize=20, ha='left', transform=skew.ax.get_yaxis_transform())
# ---------------------#


# VERTICAL VELOCITY GRAPH (m/s)
# ---------------------#
# Set up axes
ax1 = plt.axes((0.255,0,0.1,1), sharey=skew.ax)
w_filt = w[::2]                                         # Use every second data point for vertical velocity
cc = [('red' if p > 0 else 'blue') for p in w_filt]     # Red bars if upward, blue bars if downward
ax1.barh(P[::2], w_filt, align='edge', color=cc)
ax1.plot(w,P, color='purple', lw=0.4)
ax1.axvline(x=0, color='k', lw=0.4)
ax1.set_xlim([-0.2,0.5])
ax1.axes.get_yaxis().set_visible(False)

# Modify Graph Characteristics
ax1.set(facecolor='None')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.tick_params(axis='x', pad=15)
ax1.set_ylim(1030,200)
# ---------------------#


# Hodograph
# ---------------------#
# ---------------------#
ax2 = plt.axes((0.65,0.4,0.6,0.6))
hodo = Hodograph(ax2, component_range=100)
hodo.add_grid(increment=10, color='grey', alpha=0.67, zorder=0)
ax2.set_xlim(-60,60)
ax2.set_ylim(-60,60)
plt.xticks(np.arange(0,0,1))
plt.yticks(np.arange(0,0,1))
for i in range(10,80,10):
    plt.text(0,i,str(i),verticalalignment='bottom',horizontalalignment='left',fontsize=9,alpha=1,color='grey',clip_on=True)
for i in range(10,80,10):
    plt.text(i,0,str(i),verticalalignment='bottom',horizontalalignment='left',fontsize=9,alpha=1,color='grey',clip_on=True)

hodo_color = ['magenta','red','limegreen','gold','lightblue']


# Calculates storm motions
rm,lm,mw = mpcalc.bunkers_storm_motion(P,uwnd,vwnd,height)[:]
rmu = rm[0]
rmv = rm[1]
lmu = lm[0]
lmv = lm[1]
mwu = mw[0]
mwv = mw[1]

ax2.plot(rmu,rmv, color='red',marker='s',markersize=5,zorder=5,clip_on=True)
ax2.plot(lmu,lmv, color='blue',marker='s',markersize=5,zorder=5,clip_on=True)
ax2.plot(mwu,mwv, color='brown',marker='s',markersize=5,zorder=5,clip_on=True)

# Layers
h1 = max(np.where(height<=1000.*units('meter'))[0])
h3 = max(np.where(height<=3000.*units('meter'))[0])
h6 = max(np.where(height<=6000.*units('meter'))[0])
h9 = max(np.where(height<=9000.*units('meter'))[0])
h12 = max(np.where(height<=12000.*units('meter'))[0])
hh = [1,3,6,9,12]

ax2.plot(uwnd[0:h1+1], vwnd[0:h1+1], color=hodo_color[0], linewidth=2, clip_on=True)
ax2.plot(uwnd[h1:h3+1], vwnd[h1:h3+1], color=hodo_color[1], linewidth=2, clip_on=True)
ax2.plot(uwnd[h3:h6+1], vwnd[h3:h6+1], color=hodo_color[2], linewidth=2, clip_on=True)
ax2.plot(uwnd[h6:h9+1], vwnd[h6:h9+1], color=hodo_color[3], linewidth=2, clip_on=True)
ax2.plot(uwnd[h9:h12+1], vwnd[h9:h12+1], color=hodo_color[4], linewidth=2, clip_on=True)

for i in (h1,h3,h6,h9,h12):
    hodo.plot(uwnd[i],vwnd[i],color='black',marker='o',markersize=2.5,clip_on=True)
for i in (h1,h3,h6,h9,h12):
    ax2.annotate(str(int((height[i]/1000).round())),(uwnd[i].values,vwnd[i].values),xytext=(0,7),textcoords='offset pixels',horizontalalignment='center',clip_on=True) 

# Map Display
# ---------------------#
# ---------------------#
ax3 = plt.axes((0.65,0.0,0.6,0.38),projection=ccrs.TransverseMercator(central_longitude=21, false_easting=1500000))
ax3.set_extent((lon_s-9, lon_s+9, lat_s-2.5, lat_s+2.5))

# Establish Features
lakes = cfeature.NaturalEarthFeature('physical', 'lakes', '10m', edgecolor='k', facecolor='none')
# Plot Features
ax3.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.7)
ax3.add_feature(cfeature.BORDERS, linewidth=0.7)
globe = ccrs.Globe(ellipse='GRS80')
data_crs = ccrs.TransverseMercator(false_easting=500000, false_northing=0, 
                                   central_longitude=27.0, central_latitude=0, 
                                   scale_factor=0.9996, globe=globe)
ax3.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='black', linewidths=0.7, alpha=0.88)
ax3.add_geometries(roadshp['geometry'], crs=data_crs, facecolor='None',edgecolor='gold', linewidths=0.4, alpha=1)

# Build Colormap
# --
color_0050 = plt.cm.Greys(np.linspace(0.2,0.7,43))
color_50100 = plt.cm.Blues(np.linspace(0.2,0.7,42))
color_50300 = plt.cm.plasma_r(np.linspace(0,1,171))
color_cp = np.vstack((color_0050, color_50100, color_50300))
map_cp = mcolors.LinearSegmentedColormap.from_list('my_colormap', color_cp)
# --     
mesh = ax3.pcolormesh(cape.x, cape.y, cape, transform = cape.metpy.cartopy_crs, cmap=map_cp, vmin=0, vmax=3000)
plt.scatter(lon_s, lat_s, s=40, color='red', transform=ccrs.PlateCarree())
fig.text(1.04,0.34, 'MUCAPE', size=25, weight='bold')
plt.colorbar(mesh, orientation='horizontal',fraction=0.026,pad=0.0,aspect=60,extendrect=True)

# Titles
skew.ax.set_title(f"MEPS Sounding {lat_s:.3f}N {lon_s:.3f}E", size=15, loc='left')
skew.ax.set_title('Init: '+((FF00.dt.strftime('%Y-%m-%d %HZ')).values)+' | Valid '+str(Year)+'-'+str(Month)+'-'+str(Day)+'_'+str(Time)+'z', size=15, loc='right')

# Watermark
fig.text(0.66, 0.95, '@FIN_storm', size=25, alpha=0.3,weight='bold', 
     horizontalalignment='center',
     verticalalignment='center')